Analyse the documents that were retrieved from OpenAlex.

In [4]:
import json
from os import listdir

In [12]:
docs_folder = '../data/openalex/docs'
subjects_file = '../data/openalex/subjects.json'
subjects = json.load(open(subjects_file))

In [7]:
doc_cnt = {}  # map subjects to their number of docs
for file in listdir(docs_folder):
  docs = json.load(open(f'{docs_folder}/{file}'))
  for subject in docs:
    doc_cnt[subject] = len(docs[subject])
len(doc_cnt)

2157

In [14]:
sum(doc_cnt.values())  # total no. of docs

215203

In [9]:
missing_cnt = {}  # no. of subjects that don't have 100 docs
for subject, cnt in doc_cnt.items():
  if cnt != 100:
    missing_cnt[subject] = cnt
len(missing_cnt)

7

In [13]:
for subject, cnt in missing_cnt.items():
  print(subjects[subject]['name'], cnt)

Exploit 8
Algorithm design 0
Central nervous system disease 45
Social system 62
Scrutiny 86
Premise 0
Strengths and weaknesses 2
